In [48]:
#Variables
R.<x> = ZZ[]
C = ComplexField(10000)
p = 2
n = 1
q = p^n
dim = 4
deg = 2 * dim

#Change p,n in variables and below in the loads to your new value of q = p^n, up to q = 4 if you want the whole file to be able to run
#can use any q if you want to run everything except the last parts after def CharpolysofsimpleAVofdim4frombounds(l,p,n):

#change 2 in the load the below to match the new value of q, so that it is load("q-weil_polynomials_dim4.sage")

load("2-weil_polynomials_dim4.sage")

#the file that has been loaded contains the characteristic polynomials of all simple abelian varieties over F_q of dimension 4
#this data has been downloaded from the LMFDB and the polynomials have been separated based on p-rank
#The LMFDB-data uses L-polynomials (which are Weil polynomials, but with the coefficients reversed)
#Hence we convert each element in the lists to Weil polynomials

def ConvertLtoWeil(LMFDBlist):
    Weilpolys = []
    for polynomials in LMFDBlist:
        d = polynomials.degree()
        newpolycoeffs = []
        for i in range(d, -1, -1):
            newpolycoeffs.append(polynomials[i])
        Weilpolys.append(R(newpolycoeffs))
    return Weilpolys

charpolydim4prank4 = ConvertLtoWeil(data_dim4_prank4)
charpolydim4prank3 = ConvertLtoWeil(data_dim4_prank3)
charpolydim4prank2 = ConvertLtoWeil(data_dim4_prank2)
charpolydim4prank1 = ConvertLtoWeil(data_dim4_prank1)
charpolydim4prank0 = ConvertLtoWeil(data_dim4_prank0)
charpolydim4 = charpolydim4prank4 + charpolydim4prank3 + charpolydim4prank2 + charpolydim4prank1 + charpolydim4prank0

#data_dim_4_prankm is the list of data of simple abelian varieties of dimension 4 over F_q with p-rank m

In [39]:
#Built-in function in Sage to generate all q-Weil polynomials of degree deg
L = R.weil_polynomials(deg, q)

In [40]:
#Generate list of all possible coefficients as in Theorem 10.2.3

def Weilpolysofdeg8frombounds(q):
    l = []

    bounda1lower = math.floor(- 8 * sqrt(q)) + 1
    bounda1higher = math.ceil(8 * sqrt(q))

    for a1 in range(bounda1lower, bounda1higher):
        u2part1 = 3/8 * a1^2 + 4 * q
    
        bounda2lower = math.floor(6 * sqrt(q) * abs(a1) - 20 * q) + 1
        bounda2higher = math.floor(u2part1) + 1
    
        for a2 in range(bounda2lower, bounda2higher):
            u2 = (-u2part1 + a2)/2
        
            u3part1 = a1 * a2 / 2 - a1^3 / 8 + a1 * q
            bounda3firstvar = (-4/3 * u2)^(3/2)
        
            bounda3lower1 = math.ceil(u3part1 - bounda3firstvar)
            bounda3higher1 = math.floor(u3part1 + bounda3firstvar) + 1
        
            bounda3secondabs = 4 * sqrt(q) * a2 + 16 * q * sqrt(q)
            bounda3secondvar = - 9 * q * a1
        
            bounda3lower2 = math.floor(- bounda3secondabs + bounda3secondvar) + 1
            bounda3higher2 = math.ceil(bounda3secondabs + bounda3secondvar)
        
            bounda3lower = max(bounda3lower1, bounda3lower2)
            bounda3higher = min(bounda3higher1, bounda3higher2)
        
            for a3 in range(bounda3lower, bounda3higher):
                u3 = (u3part1 - a3) / 4
            
                bounda4abs = 3/256 * a1^4 - a1^2 * a2 / 16 - a1^2 * q / 2 + a1 * a3 / 4 + 2 * a2 * q - 2 * q^2 + 2/3 * u2^2
            
                etareal = - u2^6 / 27 - 5 * u2^3 * u3^2 + 27/2 * u3^4
                etaimaginary = 27/2 * u3 * (- u3^2 - 4/27 * u2^3)^(3/2)
            
                base = C(etareal, etaimaginary)
                eta = base.nth_root(3)
                eta2 = eta * E(3)
                eta3 = eta * E(3)^2
            
                root1 = 2 * real_part(eta)
                root2 = 2 * real_part(eta2)
                root3 = 2 * real_part(eta3)
            
                bounda4lower1 = math.ceil(bounda4abs + min([root1, root2, root3]))
                bounda4higher = math.floor(bounda4abs + median([root1, root2, root3])) + 1
            
                bounda4lower2 = math.floor(2 * sqrt(q) * abs(a1 * q + a3) - 2 * q * a2 - 2 * q^2) + 1
            
                bounda4lower = max(bounda4lower1, bounda4lower2)
            
                for a4 in range(bounda4lower, bounda4higher):
                    l.append(R([q^4, a1 * q^3, a2 * q^2, a3 * q, a4, a3, a2, a1, 1]))
                
    return(l)

l = Weilpolysofdeg8frombounds(q)

In [41]:
#Check for differences between lists generated by the built-in function and the one generated by Theorem 10.2.3
def comparisonlists(l,L):
    Missing = []

    for e in L:
        if e not in l:
            Missing.append(e)
            print(e.factor())

    print("number of polynomials missed by the bounds:", len(Missing))

    extra = []
    for f in l:
        if f not in L:
            extra.append(f)
            print(f.factor())

    print("number of extra polynomials found by the bounds", len(extra))

    Missingreals = []

    for pols in Missing:
        if (pols(sqrt(q)) == 0) or (pols(- sqrt(q)) == 0):
            Missingreals.append(pols)

    print("number of the missing polynomials that have a real root:", len(Missingreals))

    realsinresult = []

    for realtest in l:
        if (realtest(sqrt(q)) == 0) or (realtest(- sqrt(q)) == 0):
            realsinresult.append(realtest)

    print("number of weil polynomials with real roots satisfying the bounds:", len(realsinresult))

comparisonlists(l,L)

(x^2 - 2)^2 * (x^2 + 2*x + 2)^2
(x^2 + x + 2) * (x^2 + 2*x + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 + 3*x^3 + 5*x^2 + 6*x + 4)
(x^2 - 2)^2 * (x^2 + x + 2)^2
(x^2 + 2) * (x^2 + 2*x + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 + 2*x^3 + 3*x^2 + 4*x + 4)
(x^2 - 2)^2 * (x^4 + 2*x^3 + 2*x^2 + 4*x + 4)
(x^2 + 2) * (x^2 + x + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 + x^3 + 3*x^2 + 2*x + 4)
(x^2 - x + 2) * (x^2 + 2*x + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 + x^3 + x^2 + 2*x + 4)
(x^2 - 2)^2 * (x^4 + x^3 + 2*x + 4)
(x^2 - 2)^2 * (x^4 + x^3 - x^2 + 2*x + 4)
(x^2 - 2)^2 * (x^2 + 2)^2
(x^2 - x + 2) * (x^2 + x + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 + 2*x^2 + 4)
(x^2 - 2)^2 * (x^4 + x^2 + 4)
(x^2 - 2*x + 2) * (x^2 + 2*x + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 - x^2 + 4)
(x^2 - 2)^2 * (x^4 - 2*x^2 + 4)
(x^2 - 2)^2 * (x^4 - 3*x^2 + 4)
(x^2 - 2)^4
(x^2 - x + 2) * (x^2 + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 - x^3 + 3*x^2 - 2*x + 4)
(x^2 - 2*x + 2) * (x^2 + x + 2) * (x^2 - 2)^2
(x^2 - 2)^2 * (x^4 - x^3 + x^2 - 2*x + 4)
(x^2

In [10]:
#The polynomials missing are exactly the ones with a real root, as described in Theorem 10.2.2 (and 10.2.1 if q is a square)

In [49]:
#check which Weil polynomials are characteristic polynomials of simple abelian varieties over F_q, according to Theorem 10.4

def CharpolysofsimpleAVofdim4frombounds(l,p,n):

    prank0 = []
    prank1 = []
    prank2 = []
    prank3 = []
    prank4 = []

    Rp = Zp(p)
    vp = Rp.valuation()
    S.<y> = Zp(p, prec = 100, type = 'capped-rel', print_mode = 'series')[]

    for polyns in l:
        if polyns.is_irreducible():
            d = polyns.degree()
            vpa4 = vp(polyns[d - 4])
            vpa3 = vp(polyns[d - 3])
            vpa2 = vp(polyns[d - 2])
            vpa1 = vp(polyns[d - 1])
        
            if vpa4 == 0:
                prank4.append(polyns)
            else:
                rootsof = polyns.roots(Rp)
                rv = []
                if not len(rootsof) == 0:
                    for roots in rootsof:
                        rv.append(vp(roots[0]))
            
                coeffsinZp = []
                for co in range(polyns.degree() + 1):
                    coeffsinZp.append(polyns[co])
                    polynsinZp = S(coeffsinZp)
            
                factorinZp = polynsinZp.factor()
                fd = []
                for fa in factorinZp:
                    fd.append(fa[0].degree())
            
                if vpa4 >= n/2 and vpa3 == 0 and n/2 not in rv:
                    prank3.append(polyns)
            
                elif vpa4 >= n and vpa3 >= n/2 and vpa2 == 0 and n/2 not in rv:
                    prank2.append(polyns)
            
                elif vpa4 == n and vpa3 >= 2/3 * n and vpa2 >= n/3 and vpa1 == 0 and (2/3 * n not in rv) and (n/3 not in rv):
                    prank1.append(polyns)
            
                elif vpa4 >= 3/2 * n and vpa3 >= n and vpa2 >= n/2 and vpa1 == 0 and (n/2 not in rv) and 3 not in fd:
                    prank1.append(polyns)            

                elif vpa4 == n and vpa3 >= 3/4 * n and vpa2 >= n/2 and vpa1 >= n/4 and len(rv) == 0 and 2 not in fd:
                    prank0.append(polyns) 

                elif vpa4 >= 3/2 * n and vpa3 == n and vpa2 >= 2/3 *n and vpa1 >= n/3 and len(rv) == 0:
                    prank0.append(polyns) 
            
                elif vpa4 >= 2 * n and vpa3 >= 3/2 * n and vpa2 >= n and vpa1 >= n/2 and len(rv) == 0 and 3 not in fd:
                    prank0.append(polyns) 
                

        elif len(polyns.factor()) == 1:
            if n % 4 == 0 and polyns.factor()[0][1] == 4:
                omega = polyns.factor()[0][0][1]
                if abs(omega) < 2 * sqrt(q) and omega % q^(1/4) == 0:
                    if not omega / q^(1/4) % p == 0:
                        prank0.append(polyns)
        
            elif polyns.factor()[0][1] == 2:
                m = polyns.factor()[0][0]
                d = m.degree()
                b2 = m[d - 2]
                b1 = m[d - 1]
            
                vpb2 = vp(b2)
                vpb1 = vp(b1)
            
                rootsof = m.roots(Rp)
                rv = []
                if not len(rootsof) == 0:
                    for roots in rootsof:
                        rv.append(vp(roots[0]))
            
                coeffsinZp = []
                for co in range(m.degree() + 1):
                    coeffsinZp.append(m[co])
                    polynsinZp = S(coeffsinZp)
            
                factorinZp = polynsinZp.factor()
                fd = []
                for fa in factorinZp:
                    fd.append(fa[0].degree())
            
                if vpb2 >= n/2 and vpb1 == 0 and Rp((b2 + 2 * q)^2 - 4 * q * b1).is_square():
                    prank2.append(polyns)
            
                elif vpb2 == n/2 and vpb1 >= n/4 and len(rv) == 0:
                    prank0.append(polyns)
                
                elif vpb2 >= n and vpb1 >= n/2 and not len(rv) == 0:
                    prank0.append(polyns)

    total = prank0 + prank1 + prank2 + prank3 + prank4
    
    return prank4, prank3, prank2, prank1, prank0, total

prank4, prank3, prank2, prank1, prank0, total = CharpolysofsimpleAVofdim4frombounds(l,p,n)

#beneath is the comparison of characteristic polynomials of simple abelian varieties of dimension 4 over F_q with data from LMFDB

In [50]:
Missing4 = []

for m in charpolydim4prank4:
    if m not in prank4:
        Missing4.append(m)

extra4 = []

for M in prank4:
    if M not in charpolydim4prank4:
        extra4.append(M)

print("number of characteristic polynomials p-rank 4 in LMFDB:", len(data_dim4_prank4))
print("number of characteristic polynomials p-rank 4 according to code:", len(prank4))
print("characteristic polynomials in LMFDB but not in code:", Missing4)
print("characteristic polynomials in code, but not in LMFDB:", extra4)

number of characteristic polynomials p-rank 4 in LMFDB: 453
number of characteristic polynomials p-rank 4 according to code: 453
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [51]:
Missing3 = []

for m in charpolydim4prank3:
    if m not in prank3:
        Missing3.append(m)

extra3 = []

for M in prank3:
    if M not in charpolydim4prank3:
        extra3.append(M)

print("number of characteristic polynomials p-rank 3 in LMFDB:", len(data_dim4_prank3))
print("number of characteristic polynomials p-rank 3 according to code:", len(prank3))
print("characteristic polynomials in LMFDB but not in code:", Missing3)
print("characteristic polynomials in code, but not in LMFDB:", extra3)

number of characteristic polynomials p-rank 3 in LMFDB: 94
number of characteristic polynomials p-rank 3 according to code: 94
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [52]:
Missing2 = []

for m in charpolydim4prank2:
    if m not in prank2:
        Missing2.append(m)

extra2 = []

for M in prank2:
    if M not in charpolydim4prank2:
        extra2.append(M)

print("number of characteristic polynomials p-rank 3 in LMFDB:", len(data_dim4_prank2))
print("number of characteristic polynomials p-rank 3 according to code:", len(prank2))
print("characteristic polynomials in LMFDB but not in code:", Missing2)
print("characteristic polynomials in code, but not in LMFDB:", extra2)

number of characteristic polynomials p-rank 3 in LMFDB: 43
number of characteristic polynomials p-rank 3 according to code: 43
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [53]:
Missing1 = []

for m in charpolydim4prank1:
    if m not in prank1:
        Missing1.append(m)

extra1 = []

for M in prank1:
    if M not in charpolydim4prank1:
        extra1.append(M)

print("number of characteristic polynomials p-rank 3 in LMFDB:", len(data_dim4_prank1))
print("number of characteristic polynomials p-rank 3 according to code:", len(prank1))
print("characteristic polynomials in LMFDB but not in code:", Missing1)
print("characteristic polynomials in code, but not in LMFDB:", extra1)

number of characteristic polynomials p-rank 3 in LMFDB: 32
number of characteristic polynomials p-rank 3 according to code: 32
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [54]:
Missing0 = []

for m in charpolydim4prank0:
    if m not in prank0:
        Missing0.append(m)

extra0 = []

for M in prank0:
    if M not in charpolydim4prank0:
        extra0.append(M)

print("number of characteristic polynomials p-rank 3 in LMFDB:", len(data_dim4_prank0))
print("number of characteristic polynomials p-rank 3 according to code:", len(prank0))
print("characteristic polynomials in LMFDB but not in code:", Missing0)
print("characteristic polynomials in code, but not in LMFDB:", extra0)

number of characteristic polynomials p-rank 3 in LMFDB: 43
number of characteristic polynomials p-rank 3 according to code: 43
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [55]:
Missingtotal = Missing0 + Missing1 + Missing2 + Missing3 + Missing4
extratotal = extra0 + extra1 + extra2 + extra3 + extra4

print("total number of characteristic polynomials in LMFDB:", len(charpolydim4))
print("total number of characteristic polynomial according to code:", len(total))
print("characteristic polynomials in LMFDB but not in code:", Missingtotal)
print("characteristic polynomials in code but not in LMFDB:", extratotal)

total number of characteristic polynomials in LMFDB: 665
total number of characteristic polynomial according to code: 665
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code but not in LMFDB: []
